In [ ]:
import sys, csv
from modules.wordTokenizer import WordTokenizer
from modules.charTokenizer import CharTokenizer
from modules.LSTMCRFNetwork import LSTMCRFNetwork
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
import pandas as pd

In [ ]:
MAX_TEXTS = 1000000
#Read product file
texts, tags, brands = [], [], []
with open("data/train_products.csv", 'r') as f:
    reader = csv.DictReader(f, fieldnames=["title","brand","tags"])
    count = 0
    for row in reader:
        #print(row)
        count += 1
        text, tag_set = row['title'], row['tags'].split(' ')[:-1]
        texts.append(text)
        tags.append(tag_set)
        brands.append(row['brand'])
        if count >= MAX_TEXTS:
            break

In [ ]:
#Create training examples
numSentences = len(texts)
np.random.seed(0)
training_examples = np.random.binomial(1, 0.7, numSentences)

In [ ]:
#Separate data in training and test
trainData = []
testData  = []

nerLabels_train =[]
nerLabels_test = []


for example in range(numSentences):
    if training_examples[example] == 1:
        trainData.append(texts[example])
        nerLabels_train.append(tags[example])
    else:
        testData.append(texts[example])
        nerLabels_test.append(tags[example])

In [ ]:
#create subset of train data
numTrainSentences = 40000

trainData_tiny       = trainData[0:numTrainSentences]
nerLabels_train_tiny = nerLabels_train[0:numTrainSentences]

In [ ]:
#Initiate network
network = LSTMCRFNetwork()

In [ ]:
#Create tokenized sentences
wordTokenizer = WordTokenizer(30,'models/word_tokenizer')
wordTokenizer.load()
data = wordTokenizer.tokenize(trainData_tiny)
testData = wordTokenizer.tokenize(testData)

In [ ]:
#Generate pad labels for network
labels = network.get_labels(nerLabels_train_tiny,wordTokenizer)
testLabels = network.get_labels(nerLabels_test,wordTokenizer)

In [ ]:
network.compile(wordTokenizer)

In [ ]:
network.train(data, labels, epochs=8, validation_split=0.2, batch_size=32)

In [ ]:
network.evaluate(testData,testLabels)